In [29]:
import config
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import cv2

In [ ]:
with open(r'tmp\teste_2022-10-01-8-46-46.txt', encoding="utf8") as file:
    reader = file.readlines()
reader

In [17]:
codesPergamun = list()
for line in reader[1:]:
    codesPergamun.append(line.split('\t', 1)[0])

In [ ]:
codesPergamun

In [19]:
import httpx

In [31]:
from driver import StartDriver

In [32]:
url = "http://pergamum.museu-goeldi.br/pergamum/biblioteca/index.php?codAcervo=35592"
chrome = StartDriver(url)

In [ ]:
getMarc = chrome.find_element(By.XPATH, '//*[@id="div_detalhes_marc"]/fieldset/a[4]')

In [50]:
getMarc = chrome.find_element(By.TAG_NAME, 'fieldset').find_elements(By.TAG_NAME, 'a')[3]

In [51]:
getMarc.click()

In [78]:

tbody = chrome.find_element(By.XPATH, '//*[@id="div_detalhes_marc"]/div/table/tbody')


In [86]:
trs = tbody.find_elements(By.TAG_NAME, 'tr')

In [91]:
trs[0].find_elements(By.TAG_NAME, 'td')[0].text

'001'

In [ ]:
def GetMarc(itemID):
    url = f"http://pergamum.museu-goeldi.br/pergamum/biblioteca/index.php?codAcervo={itemID}"
    driver = StartDriver(url)
    getMarc = chrome.find_element(By.XPATH, '//*[@id="div_detalhes_marc"]/fieldset/a[4]')
    
    

In [105]:
import config
import json

In [109]:
def ParserMarc(trs):

    controfield = {}
    datafield =  {}
    for tr in trs:
        tds = tr.find_elements(By.TAG_NAME, 'td')
        field = tds[0].text
        subfield = tds[2].text
        if int(field) < 9:
            controfield[field] = subfield
        else:
            subfields = {}
            subfield = subfield.split('$')[1:]
            for sub in subfield:
                code = sub.split(' ', 1)[0]
                value = sub.split(' ', 1)[1]
                subfields[code] = value
            if field in config.MARC_REPETIVEL:
                if field not in datafield.keys():
                    datafield[field] = [subfields]
                else:
                    datafield[field].append(subfields)
            else:
                datafield[field] = subfields

    marcJson = {'controfield': controfield, 'datafield': datafield}
    return marcJson


In [ ]:
marcJson = ParserMarc(trs)
marcJson

In [116]:
itemID = marcJson['controfield']['001']
with open(
        f"out\goeldi/01-09-22_30-09-22/{itemID}.json","w", encoding='utf-8') as jsonfile:
        json.dump(marcJson, jsonfile, ensure_ascii=False, indent=4)
        jsonfile.close()

In [114]:
marcJson['controfield']['001']

'35592'